In [1]:
# headers = {
# 		"Content-Type": "application/json",
# 		"Connection": "keep-alive",
# 		"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:120.0) Gecko/20100101 Firefox/120.0",
# 		"Access-Control-Allow-Origin": "*"
# }

# import re

# def remove_whitespace(s: str, keep_spaces: bool = True) -> str:
#     """
#     Clean whitespace while keeping natural spaces between words.

#     - Convert all non-space whitespace (tabs, newlines, etc.) into a space
#     - Collapse multiple spaces into one
#     - Trim leading/trailing spaces
#     """
#     # Replace tabs/newlines/etc. with a space
#     s = re.sub(r"[^\S ]+", " ", s)

#     # Collapse multiple spaces into one
#     s = re.sub(r" +", " ", s)
    
#     # Replace \ by ""
#     s = re.sub(r"\\", "", s)
    
#     # Trim
#     return s.strip()


# def get_specific_element(regex: str, text: str):
#     m = re.search(regex, text)
#     return m.group(0) if m else None


# def safe_text(element, keep_spaces=False):
#     if not element:
#         return ""
#     try:
#         e = element.text
#     except AttributeError:
#         e = element
#     return remove_whitespace(e, keep_spaces)







In [2]:
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.action_chains import ActionChains
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from bs4 import BeautifulSoup
# from time import sleep


# def scrape_annonce(url, htmlContent):
#     """
#     Scrapes a ParuVendu annonce page:
#     - Opens Selenium
#     - Accepts cookies
#     - Clicks `Lire plus` when present
#     - Extracts description using Selenium
#     - Parses remaining fields using BeautifulSoup
#     """
    
#     # ---------- SELENIUM ----------
#     opt = Options()
#     opt.add_argument("start-maximized")
#     opt.add_argument("--lang=en-US")

#     driver = webdriver.Chrome(
#         service=Service(ChromeDriverManager().install()),
#         options=opt
#     )

#     driver.get(url)
#     sleep(2)

#     try:
#         btn = driver.find_element(By.XPATH, "//button[text()='Accepter']")
#         driver.execute_script("arguments[0].click();", btn)
#         sleep(1)
#     except:
#         pass  

#     # Click "Lire plus"
#     try:
#         read_more = driver.find_element(By.ID, "linkAnnonceTrunc")
#         driver.execute_script("arguments[0].click();", read_more)
#         sleep(2)
#     except Exception as e:
#         print(f"[⚠] Lire plus not found: {e}")

#     # Extract description
#     try:
#         element = driver.find_element(
#             By.XPATH, "//div[@class='im12_txt_ann im12_txt_ann_auto']"
#         )
#         description = element.text
#     except:
#         description = ""

#     driver.quit()

#     # ---------- BEAUTIFULSOUP PART ----------
#     data = {}

#     if htmlContent:
#         soup = BeautifulSoup(htmlContent, 'html.parser')

#         data["title"] = safe_text(
#             soup.find("span", {"id": "detail_h1"}), True
#         )

#         data["location"] = safe_text(
#             soup.find("span", {"id": "detail_loc"})
#         )

#         data["descriptionTitle"] = safe_text(
#             soup.find("h2", {"class": "autodetail-titre sepdetail14-ssbordure"})
#         )

#         data["description"] = safe_text(description)

#         data["nbp"] = get_specific_element(
#             r"\d+",
#             safe_text(soup.find("li", {"class": "nbp"}))
#         )

#         data["surf"] = get_specific_element(
#             r"\d+",
#             safe_text(soup.find("li", {"class": "surf"}))
#         )

#         data["prix"] = get_specific_element(
#             r"\d+(?: \d+)*",
#             safe_text(soup.find("div", {"class": "prixactionalerte-box"}))
#         )

#         data["enseigneInfosvendeur"] = safe_text(
#             soup.find("p", {"class": "enseigne-infosvendeur"})
#         )

#     return data


In [3]:
# import requests
# import os 


# annonces_link_csv = "annonces_link.csv"

# if os.path.exists(annonces_link_csv):
#     with open(annonces_link_csv, "r") as f:
#         links = f.readlines()
#         for link in links :  
#             annonceUrl = remove_whitespace(link)

#             print("On charge le site : " + link)
#             response = requests.get(annonceUrl,headers=headers)
            
#             content = ""
            
    
#             if response.status_code == 200:
#                 print("Site web chargé")
#                 content = response.text
#             else:
#                 print("Erreur lors du chargement de la page , Status Code : "+response.status_code)
        
                
#             result = scrape_annonce(
#                 url=annonceUrl,
#                 htmlContent=content)

#             print(result)

        



In [4]:
import os
import csv
import re
import requests
from time import sleep
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

# -------------------- CONFIG --------------------
headers = {
    "Content-Type": "application/json",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:120.0) Gecko/20100101 Firefox/120.0",
    "Access-Control-Allow-Origin": "*"
}

input_csv = "annonces_link.csv"    
output_csv = "results.csv"          

# -------------------- UTILITIES --------------------
def remove_whitespace(s: str, keep_spaces: bool = True) -> str:
    if not s:
        return ""
    s = re.sub(r"[^\S ]+", " ", s)
    s = re.sub(r" +", " ", s)
    s = re.sub(r"\\", "", s)
    return s.strip()

def get_specific_element(regex: str, text: str):
    if not text:
        return None
    m = re.search(regex, text)
    return m.group(0) if m else None

def safe_text(element, keep_spaces=False):
    if not element:
        return ""
    try:
        e = element.text
    except AttributeError:
        e = element
    return remove_whitespace(e, keep_spaces)

def stringToTimeStamp(date_str:str):
    dt = datetime.strptime(date_str, "%d/%m/%Y %H:%M")
    return int(dt.timestamp())
    

def safe_find_text(soup, tag, attrs=None, keep_spaces=False):
    element = soup.find(tag, attrs=attrs)
    return safe_text(element, keep_spaces) if element else ""

def scrape_annonce(url, htmlContent):
    """
    Scrapes a ParuVendu annonce page:
    - Selenium: handles dynamic content & clicks 'Lire plus'
    - BeautifulSoup: parses static content
    """
    # ---------- SELENIUM ----------
    opt = Options()
    opt.add_argument("start-maximized")
    opt.add_argument("--lang=en-US")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=opt
    )

    driver.get(url)
    sleep(2)

    # Accept cookies
    try:
        btn = driver.find_element(By.XPATH, "//button[text()='Accepter']")
        driver.execute_script("arguments[0].click();", btn)
        sleep(1)
    except:
        pass

    # Cliquer sur  "Lire plus" si ca exite 
    try:
        read_more = driver.find_element(By.ID, "linkAnnonceTrunc")
        driver.execute_script("arguments[0].click();", read_more)
        sleep(1)
    except:
        pass

    # Extraire la description
    try:
        element = driver.find_element(By.XPATH, "//div[@class='im12_txt_ann im12_txt_ann_auto']")
        description = element.text
    except:
        description = ""

    driver.quit()

    # ---------- BEAUTIFULSOUP ----------
    data = {}
    if htmlContent:
        soup = BeautifulSoup(htmlContent, 'html.parser')
        data["title"] = safe_find_text(soup, "span", {"id": "detail_h1"}, True)
        data["location"] = safe_find_text(soup, "span", {"id": "detail_loc"})
        data["descriptionTitle"] = safe_find_text(soup, "h2", {"class": "autodetail-titre sepdetail14-ssbordure"})
        data["description"] = safe_text(description)
        data["nbp"] = get_specific_element(r"\d+", safe_find_text(soup, "li", {"class": "nbp"}))
        data["surf"] = get_specific_element(r"\d+", safe_find_text(soup, "li", {"class": "surf"}))
        data["prix"] = get_specific_element(r"\d+(?: \d+)*", safe_find_text(soup, "div", {"class": "prixactionalerte-box"}))
        data["enseigneInfosvendeur"] = safe_find_text(soup, "p", {"class": "enseigne-infosvendeur"})
        # data["enseigneInfosvendeur"] = safe_find_text(soup, "p", {"class": "enseigne-infosvendeur"})

        uls = soup.find_all("ul", {"class": "crit-alignbloc"})
        
        for idx, ul in enumerate(uls, 1):
            datas = {}
            li_items = ul.find_all("li")
        
            for li in li_items:
        
                strong = li.find("strong")
                if strong:  
                    # category is inside <strong>
                    category = strong.get_text(strip=True)
                else:
                    # category is plain text before spans
                    category = li.get_text(" ", strip=True).split(" ")[0]
        
                values = [span.get_text(strip=True) for span in li.find_all("span")]
                datas[category] = values
        print(datas)
        
        data["agencement"] = ",".join(datas.get("Agencement", []))
        data["general"] = ",".join(datas.get("Général", []))
        data["annexe"] = ",".join(datas.get("Annexes", []))

        data["dependance"] = ",".join(datas.get("Dépendance", []))

        # Tranformer la date en timestamp
        raw_date = datas.get("Mise", [])
        if raw_date:
            date_str = raw_date[0].replace(" à ", " ")
            data["publishedAt"] = stringToTimeStamp(date_str)
        else:
            data["publishedAt"] = None


        data["reference"] =  safe_text(datas.get("Réf.")[0],[])

    return data


/Users/nebel/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
link= "https://www.paruvendu.fr/immobilier/vente/maison/1283596970A1KIVHMN000"
response = requests.get(link, headers=headers)
if response.status_code == 200:
    content = response.text
    data = scrape_annonce(link, content)
    print(data)
else :
    print(f"Error loading page: {response.status_code}")


{'Agencement': ['4 chambres', 'Cheminée'], 'Général': ['Chauffage : électrique radiateur'], 'Extérieur': ['Surface  terrain : 242\r\nm2'], 'Dépendance': ['Terrasse'], 'Réf.': ['ParuVendu \t\t\t\t\t\t1789197'], 'Mise': ['18/11/2025 à 06:36']}
{'title': 'Vente Maison 8 pièces 158 m²', 'location': 'Pierrelaye (95480)', 'descriptionTitle': 'Description T8 à Pierrelaye', 'description': "Vente Maison/villa 8 pièces iad France - Fabien Pruvot vous propose: Maison de caractère à Pierrelaye, à 7 min à pied de la gare Située dans un quartier pavillonnaire très recherché, cette bâtisse aux volumes généreux ne manquera pas de vous séduire ! Édifiée sur trois niveaux de 70m² au sol et complétée par une cave voûtée de 22m², elle offre un véritable potentiel pour les amateurs d'espace. Au rez-de-chaussée surélevé, vous serez accueilli par une vaste pièce de vie lumineuse, dotée d'une cuisine semi-ouverte meublée et équipée, une chambre, une salle d'eau avec WC. À l'étage, un grand palier dessert deux

In [6]:
processed_links = set()
if os.path.exists(output_csv):
    with open(output_csv, "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        for row in reader:
            if row:
                processed_links.add(row[0].strip())

# Lire tout les links a scraper 

if os.path.exists(input_csv):
    with open(input_csv, "r", encoding="utf-8") as f:
        all_links = [line.strip() for line in f if line.strip()]
else:
    print("Input CSV not found.")
    all_links = []

# Ouvrir  results CSV en monde append (pour rajouter des lignes)
with open(output_csv, "a", encoding="utf-8", newline="") as out_file:
    writer = csv.writer(out_file)

    for idx,link in enumerate(all_links):
        if link in processed_links:
            continue  

        try:
            print(f"Processing: {link}  ({idx}/{len(all_links)})")
            response = requests.get(link, headers=headers)
            if response.status_code != 200:
                print(f"Error loading page: {response.status_code}")
                continue

            content = response.text
            data = scrape_annonce(link, content)

          
            writer.writerow([link, data])
            out_file.flush()

            sleep(2)

        except Exception as e:
            print(f"Error processing {link}: {e}")


Processing: https://www.paruvendu.fr/immobilier/vente/appartement/1282974090A1KIVHAP000  (0/30030)
Error loading page: 410
Processing: https://www.paruvendu.fr/immobilier/vente/appartement/1265798356A1KIVHAP000  (1/30030)
{'Agencement': ["1 salle d'eau"], 'Général': ['Etage : 4 (dernier étage)', "Nombre d'étage(s) dans l'immeuble : 4"], 'Accès': ['Digicode', 'Interphone', 'Concierge'], 'Annexes': ['Parking'], 'Réf.': ['ParuVendu \t\t\t\t\t\t13183462_979'], 'Mise': ['15/11/2025 à 05:15']}
Processing: https://www.paruvendu.fr/immobilier/vente/appartement/1275249674A1KIVHAP000  (2/30030)
{'Agencement': ['1 chambre', "1 salle d'eau"], 'Général': ['Etage : 6 (dernier étage)', "Nombre d'étage(s) dans l'immeuble : 6"], 'Accès': ['Ascenseur', 'Digicode', 'Interphone', 'Concierge'], 'Dépendance': ['Balcon'], 'Annexes': ['Parking'], 'Réf.': ['ParuVendu \t\t\t\t\t\t13600420_1201'], 'Mise': ['26/11/2025 à 05:15']}


KeyboardInterrupt: 

In [ ]:


# ### **Agency**

# * `id`
# * `name`
# * `address`
# * `slogan`
# * `description`
# * `telephone`
# * `announcements` (relationship)

# ---

# ### **Announcement**

# * `id`
# * `ref`
# * `title`
# * `description`
# * `price`
# * `price_per_meter`
# * `publish_at`
# * `updated_at`
# * `nb_rooms`
# * `nb_bedrooms`
# * `exclusive`
# * `dpe`
# * `location`
# * `url`
# * `type`
# * `agency_id`
# * `agency` (relationship)

# ---

# ### **Caracteristic**

# * `id`
# * `announcement_id`
# * `announcement` (relationship)
# * `parking_garage`
# * `garden`
# * `balcony_terrace`
# * `annexes`
# * `access`
# * `arrangement`
# * `dependence`
# * `outside`
# * `connectivity_index`
# * `fiber_eligibility_rate`
# * `general_information`
